# Tracing 101

Step through this notebook to understand how tracing works in Generative AI Toolkit.

The Generative AI Toolkit comes with these tracers out-of-the-box:


In [1]:
import time

from generative_ai_toolkit.tracer import (
    HumanReadableTracer,
    InMemoryTracer,
    NoopTracer,
    StructuredLogsTracer,
    TeeTracer,
    Tracer,
)
from generative_ai_toolkit.tracer.dynamodb import DynamoDbTracer
from generative_ai_toolkit.tracer.otlp import OtlpTracer
from generative_ai_toolkit.utils.ulid import Ulid

### `InMemoryTracer`

Use the in-memory tracer for testing and development:


In [2]:
in_memory_tracer = InMemoryTracer(
    memory_size=1000  # Store max 1000 traces, before discarding older ones
)

# Context, added to all traces:
in_memory_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with in_memory_tracer.trace("parent") as parent_span:
    parent_span.add_attribute("foo", "bar")
    parent_span.add_attribute(
        "inherited.foo",
        "bar",
        inheritable=True,  # Inheritable attributes propagate to child spans
    )
    time.sleep(0.1)

    # Nested spans become child spans, that point to the parent (parent_span_id):
    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("bar", "foo")
        time.sleep(0.1)

for trace in in_memory_tracer.get_traces():
    print(trace)
    print()

Trace(span_name='parent', span_kind='INTERNAL', trace_id='67a32a7c5da62a13a9697bce7ed3eb11', span_id='869f6cfef9c84be1', parent_span_id=None, started_at=datetime.datetime(2025, 4, 17, 18, 40, 4, 888534, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 5, 96727, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL', trace_id='67a32a7c5da62a13a9697bce7ed3eb11', span_id='e20697ba3a8345c5', parent_span_id='869f6cfef9c84be1', started_at=datetime.datetime(2025, 4, 17, 18, 40, 4, 992046, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 5, 96685, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)



### Printing a human-readable version of traces during development

In the following example we add attributes that Generative AI Toolkit understands. It will use these to present traces in a way that is nicer to the human eye:


In [3]:
conversation_id = Ulid().ulid

with in_memory_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in in_memory_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id  # filter traces by conversation id
    }
):
    print(trace.as_human_readable())
    print()

[b5d34de4dbe2b0fa57482f7f45107e0b/root/2356ae1dd5c2a658] MyAgent SERVER 2025-04-17T18:40:05.103Z - parent (ai.conversation.id='01JS2GT2ZF7NVKR46VQC9BKJT6' ai.auth.context='user123')


[b5d34de4dbe2b0fa57482f7f45107e0b/2356ae1dd5c2a658/ce47551f27b4cf10] MyAgent INTERNAL 2025-04-17T18:40:05.208Z - child (ai.trace.type='tool-invocation' ai.conversation.id='01JS2GT2ZF7NVKR46VQC9BKJT6' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `HumanReadableTracer`

You can also use the `HumanReadableTracer` that will log traces in human readable form to stdout, which is useful during development.

Note that traces are logged when the span ends, so parent spans are logged after child spans (this is true for all tracers):


In [4]:
import sys

human_readable_tracer = HumanReadableTracer(stream=sys.stdout)

human_readable_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with human_readable_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with human_readable_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

[11effc51d176237146ed5bdbc0c56114/01432762046b3ead/cd13e44739b4e39e] MyAgent INTERNAL 2025-04-17T18:40:05.430Z - child (ai.trace.type='tool-invocation' ai.conversation.id='01JS2GT2ZF7NVKR46VQC9BKJT6' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!

[11effc51d176237146ed5bdbc0c56114/root/01432762046b3ead] MyAgent SERVER 2025-04-17T18:40:05.325Z - parent (ai.conversation.id='01JS2GT2ZF7NVKR46VQC9BKJT6' ai.auth.context='user123')



### `StructuredLogsTracer`

Use the `StructuredLogsTracer` to log traces to stdout as JSON:


In [5]:
structured_logs_tracer = StructuredLogsTracer(stream=sys.stdout)

structured_logs_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with structured_logs_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with structured_logs_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"child","span_kind":"INTERNAL","trace_id":"c2991f68279180b89cfba134e27429da","span_id":"bc7fc0d01ca8797b","parent_span_id":"c180c46a3b6b1ace","started_at":"2025-04-17 18:40:05.650153+00:00","ended_at":"2025-04-17 18:40:05.755206+00:00","attributes":{"ai.trace.type":"tool-invocation","ai.tool.input":"Hello, world!","ai.tool.output":"World, hello!","ai.conversation.id":"01JS2GT2ZF7NVKR46VQC9BKJT6","ai.auth.context":"user123"},"span_status":"UNSET","resource_attributes":{"service.name":"MyAgent"},"scope":{"name":"generative-ai-toolkit","version":"current"}}}
{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"parent","span_kind":"SERVER","trace_id":"c2991f68279180b89cfba134e27429da","span_id":"c180c46a3b6b1ace","parent_span_id":null,"started_at":"2025-04-17 18:40:05.549289+00:00","ended_at":"2025-04-17 18:40:05.755649+00:00","attributes":{"ai.conversation.id":"01JS2GT2ZF7NVKR46VQC9BKJT

### `DynamoDbTracer`

Use the `DynamoDbTracer` to store traces to DynamoDB.

To use this tracer, you should have created a table with partition key `pk` (string) and sort key `sk` (string).

If you want to support getting traces by conversation ID, the table must have a GSI with partition key `conversation_id` (string) and sort key `sk` (string).

For example, here's how to create such a table:


In [ ]:
!aws dynamodb create-table \
  --table-name MyTracesTable \
  --attribute-definitions \
    AttributeName=pk,AttributeType=S \
    AttributeName=sk,AttributeType=S \
    AttributeName=conversation_id,AttributeType=S \
  --key-schema \
    AttributeName=pk,KeyType=HASH \
    AttributeName=sk,KeyType=RANGE \
  --billing-mode PAY_PER_REQUEST \
  --global-secondary-indexes '[{"IndexName":"conversation_index","KeySchema":[{"AttributeName":"conversation_id","KeyType":"HASH"},{"AttributeName":"sk","KeyType":"RANGE"}],"Projection":{"ProjectionType":"ALL"}}]'

Then, use that table in the `DynamoDbTracer`:


In [7]:
conversation_id = Ulid().ulid
auth_context = "user123"

ddb_tracer = DynamoDbTracer(
    table_name="MyTracesTable",
    identifier="MyAgent",
    conversation_id_gsi_name="conversation_index",
)

ddb_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with ddb_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with ddb_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in ddb_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace.as_human_readable())
    print()

[4b99255b0cfa0f5240daadb8f5c12989/root/09fb672e5eb74018] MyAgent SERVER 2025-04-17T18:40:09.253Z - parent (ai.conversation.id='01JS2GT5QH9A71AR8WP3Y87DPD' ai.auth.context='user123')


[4b99255b0cfa0f5240daadb8f5c12989/09fb672e5eb74018/d9a755f3fb6adcd8] MyAgent INTERNAL 2025-04-17T18:40:09.357Z - child (ai.trace.type='tool-invocation' ai.conversation.id='01JS2GT5QH9A71AR8WP3Y87DPD' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `OtlpTracer`

The `OtlpTracer` logs traces in Open Telemetry protobuf format. It expects you to run an Open Telemetry collector, that it can send the traces to. By default, it expects the collector to be run on localhost port 4318.

You can use the `OtlpTracer` to send traces to AWS X-Ray. To make that work, you can run the [ADOT collector](https://github.com/aws-observability/aws-otel-collector) locally:


In [8]:
# Create the ADOT config file:

yaml_content = """\
receivers:
  otlp:
    protocols:
      http:
        endpoint: 0.0.0.0:4318

processors:
  batch/traces:
    timeout: 10s
    send_batch_size: 50

exporters:
  awsxray:
    region: eu-central-1
    indexed_attributes:
      - ai.conversation.id

service:
  pipelines:
    traces:
      receivers: [otlp]
      processors: [batch/traces]
      exporters: [awsxray]
"""

with open("adot-config.yaml", "w") as f:
    f.write(yaml_content)

Run the ADOT collector in the background. Note that the following example assumes `AWS_REGION`, `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_SESSION_TOKEN` are available as environment variables:


In [ ]:
!docker run --rm -d --name adot-collector \
  -p 4318:4318 \
  -e AWS_REGION \
  -e AWS_ACCESS_KEY_ID \
  -e AWS_SECRET_ACCESS_KEY \
  -e AWS_SESSION_TOKEN \
  -v $(pwd)/adot-config.yaml:/etc/collector-config.yaml \
  public.ecr.aws/aws-observability/aws-otel-collector:latest \
  --config=/etc/collector-config.yaml
!sleep 2 # wait for the collector to start
!docker logs adot-collector

Then, send traces to AWS X-Ray by using the `OtlpTracer`:


In [10]:
otlp_tracer = OtlpTracer()

otlp_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with otlp_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", "123456", inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with otlp_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

If that seems to work, but you don't see traces appear in AWS X-Ray, check the ADOT container logs. E.g. there may be a permission issue if your AWS credentials have expired:


In [ ]:
!docker logs adot-collector

### `NoopTracer`

Use the no-operation tracer when you don't want traces:


In [12]:
noop_tracer = NoopTracer()
with noop_tracer.trace("noop") as span:
    span.add_attribute("foo", "bar")

# nothing was logged

### `TeeTracer`

Use the `TeeTracer` to send traces to multiple tracers at once.

Note that the first tracer you add, will be the one that `get_traces()` will be delegated to. So if you want to use that method, use a tracer that supports it.

Add tracers like this:


In [13]:
tee_tracer = TeeTracer()

# E.g. the DynamoDBTracer supports get_traces(), so add that first:
tee_tracer.add_tracer(ddb_tracer)

# add_tracer() can be chained for convenience:
tee_tracer.add_tracer(human_readable_tracer).add_tracer(noop_tracer)

Then, use the `TeeTracer` as any other tracer:


In [14]:
conversation_id = Ulid().ulid
auth_context = "user456"

tee_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

print("==== live traces: ====\n")

with tee_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with tee_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


print("==== traces from DynamoDB: ====\n")
for trace in tee_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace.as_human_readable())
    print()

==== live traces: ====

[3cf33cc4a29d949f302393becdb8803d/0ee353668bbee0a9/eae94b1e302ad96d] MyAgent INTERNAL 2025-04-17T18:40:13.271Z - child (ai.trace.type='tool-invocation' ai.conversation.id='01JS2GTAVHHW27ZE1VFXZT8ZJA' ai.auth.context='user456')
       Input: Hello, world!
      Output: World, hello!

[3cf33cc4a29d949f302393becdb8803d/root/0ee353668bbee0a9] MyAgent SERVER 2025-04-17T18:40:13.170Z - parent (ai.conversation.id='01JS2GTAVHHW27ZE1VFXZT8ZJA' ai.auth.context='user456')

==== traces from DynamoDB: ====

[3cf33cc4a29d949f302393becdb8803d/root/0ee353668bbee0a9] MyAgent SERVER 2025-04-17T18:40:13.170Z - parent (ai.conversation.id='01JS2GTAVHHW27ZE1VFXZT8ZJA' ai.auth.context='user456')


[3cf33cc4a29d949f302393becdb8803d/0ee353668bbee0a9/eae94b1e302ad96d] MyAgent INTERNAL 2025-04-17T18:40:13.271Z - child (ai.trace.type='tool-invocation' ai.conversation.id='01JS2GTAVHHW27ZE1VFXZT8ZJA' ai.auth.context='user456')
       Input: Hello, world!
      Output: World, hello!




### `@traced` decorator

Rather than wrapping your code inside `with` statements to add tracing, you can also you use the `@traced` decorator with your functions, to trace their execution:


In [15]:
from generative_ai_toolkit.tracer import traced

in_memory_tracer = InMemoryTracer()


@traced("parent", tracer=in_memory_tracer)
def parent_fn():
    child_fn()
    time.sleep(0.1)


@traced("child", tracer=in_memory_tracer)
def child_fn():
    time.sleep(0.1)

Now, when you execute these functions, they will be traced:


In [16]:
parent_fn()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent', span_kind='INTERNAL', trace_id='fc1fa730e4510c00a66b441dc33b6cf7', span_id='d939aa9ac6a37c94', parent_span_id=None, started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 527126, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 736427, tzinfo=datetime.timezone.utc), attributes={}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child', span_kind='INTERNAL', trace_id='fc1fa730e4510c00a66b441dc33b6cf7', span_id='d820e4e53917679e', parent_span_id='d939aa9ac6a37c94', started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 527359, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 632211, tzinfo=datetime.timezone.utc), attributes={}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


In order to add attributes to the trace, you can access the `current_trace` attribute for the tracer. Accessing that attribute only works within the context of a trace:


In [17]:
in_memory_tracer = InMemoryTracer()


@traced("parent", tracer=in_memory_tracer)
def parent_fn2():
    in_memory_tracer.current_trace.add_attribute("foo", "bar", inheritable=True)
    child_fn2()
    time.sleep(0.1)


@traced("child", tracer=in_memory_tracer)
def child_fn2():
    in_memory_tracer.current_trace.add_attribute("bar", "foo")
    time.sleep(0.1)


parent_fn2()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent', span_kind='INTERNAL', trace_id='e254f209330877090c65586b4b1b85c2', span_id='c86b85d9e8f5a177', parent_span_id=None, started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 743162, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 949445, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child', span_kind='INTERNAL', trace_id='e254f209330877090c65586b4b1b85c2', span_id='d24a697870d723ac', parent_span_id='c86b85d9e8f5a177', started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 743212, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 848286, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


If the first argument to your function has a `tracer` attribute, you don't need to specify a `tracer` explicitly. E.g. within a class with a `tracer` attribute, you can decorate methods with `@traced`, i.e. without explicitly passing the tracer, as below:


In [18]:
in_memory_tracer = InMemoryTracer()


class MyAgent:
    def __init__(self, tracer: Tracer) -> None:
        self._tracer = tracer

    @property
    def tracer(self):
        return self._tracer

    @traced
    def parent_method(self):
        self.tracer.current_trace.add_attribute("foo", "bar", inheritable=True)
        self.child_method()
        time.sleep(0.1)

    @traced
    def child_method(self):
        self.tracer.current_trace.add_attribute("bar", "foo")
        time.sleep(0.1)


agent = MyAgent(in_memory_tracer)
agent.parent_method()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent_method', span_kind='INTERNAL', trace_id='3b327625f340e89af7f0085edcdf67be', span_id='8fcfb497d3a9efdc', parent_span_id=None, started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 964760, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 14, 175250, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child_method', span_kind='INTERNAL', trace_id='3b327625f340e89af7f0085edcdf67be', span_id='5b33285f4668035b', parent_span_id='8fcfb497d3a9efdc', started_at=datetime.datetime(2025, 4, 17, 18, 40, 13, 965111, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 17, 18, 40, 14, 70238, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


### Developing your own tracer

It's easy to develop your own tracers that can be used with the Generative AI Toolkit.

In the simplest case, you inherit from `BaseTracer` and only have to implement the `persist` method:


In [19]:
from generative_ai_toolkit.tracer import BaseTracer, Trace


class MyTracer(BaseTracer):

    def persist(self, trace: Trace):
        print(trace.as_human_readable())  # This is what the `HumanReadableTracer` does


my_tracer = MyTracer()

with my_tracer.trace("span") as span:
    span.add_attribute("foo", "bar")

[174615cba5a35e8431b5cfa12bea3423/root/cdde07e75b9a4c00] <missing service.name> INTERNAL 2025-04-17T18:40:14.192Z - span

